In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn import base

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#https://www.analyticsvidhya.com/blog/2018/09/multivariate-time-series-guide-forecasting-modeling-python-codes/

In [179]:
training_data = pd.read_csv("../data/TrainingSet.csv",index_col=0)
submission_labels = pd.read_csv("../data/SubmissionRows.csv", index_col=0)


In [330]:
#df = training_data.melt(id_vars=['Series Code', 'Country Name','Series Name'], var_name='Year', value_name='indicator',ignore_index = False)
df = training_data.melt(id_vars=['Series Code', 'Country Name'], var_name='Year', value_name='indicator',ignore_index = False, value_vars=['1972 [YR1972]','1973 [YR1973]','1974 [YR1974]','1975 [YR1975]',\
                                                                                                                                          '1976 [YR1976]','1977 [YR1977]','1978 [YR1978]','1979 [YR1979]', \
                                                                                                                                          '1980 [YR1980]',  '1981 [YR1981]', '1982 [YR1982]', '1983 [YR1983]', \
                                                                                                                                          '1984 [YR1984]','1985 [YR1985]','1986 [YR1986]','1987 [YR1987]', \
                                                                                                                                          '1988 [YR1988]','1989 [YR1989]','1990 [YR1990]','1991 [YR1991]', \
                                                                                                                                          '1992 [YR1992]','1993 [YR1993]','1994 [YR1994]','1995 [YR1995]', \
                                                                                                                                          '1996 [YR1996]', '1997 [YR1997]','1998 [YR1998]','1999 [YR1999]', \
                                                                                                                                          '2000 [YR2000]','2001 [YR2001]','2002 [YR2002]','2003 [YR2003]' , \
                                                                                                                                          '2004 [YR2004]','2005 [YR2005]','2006 [YR2006]'])

df['Year'] = df['Year'].str.extract('(\d+)', expand=False) #.4stype(i4t),'2005 [YR2005]','2006 [YR2006]'

#df = df.sort_values(['Week', 'Product_Code'])

pd.set_option('display.max_colwidth', -1)

df['Year'] = pd.to_datetime(df['Year'], format='%Y')

df.fillna(0, inplace=True)


In [331]:
from __future__ import division
import pandas as pd
import numpy as np
import datetime
import re
import time
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import xgboost as xgb
from math import sqrt
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, randint
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
#import pydot_ng as pydot
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate, Dropout
from keras.layers.embeddings import Embedding
from keras.utils import plot_model
from keras.optimizers import SGD
from sklearn.model_selection import KFold, cross_val_score

In [332]:
target_encode_columns = ['Country Name','Series Code']
target = ['indicator']
target_encode_df = df[target_encode_columns + target].reset_index().drop(columns = 'index', axis = 1)
target_name = target[0]
target_df = pd.DataFrame()
for embed_col in target_encode_columns:
    val_map = target_encode_df.groupby(embed_col)[target].mean().to_dict()[target_name]
    target_df[embed_col] = target_encode_df[embed_col].map(val_map).values
    
score_target_drop = df.drop(target_encode_columns, axis = 1).reset_index().drop(columns = 'index', axis = 1)
score_target = pd.concat([score_target_drop, target_df], axis = 1)
score_target.dtypes

Year            datetime64[ns]
indicator       float64       
Country Name    float64       
Series Code     float64       
dtype: object

In [333]:
X_train, X_test,y_train,y_test = train_test_split(score_target[['Country Name','Series Code']],score_target['indicator'], test_size=0.7, random_state=42)


In [54]:
#https://www.kaggle.com/robikscube/tutorial-time-series-forecasting-with-xgboost
#not working

In [334]:

params = {'colsample_bytree': uniform(0.7, 0.3),
          'gamma': uniform(0, 0.5),
          'learning_rate': uniform(0.003, 0.1), # default 0.1 
          'max_depth': randint(2, 6), # default 3
          'n_estimators': randint(100, 250), # default 100
          'subsample': uniform(0.6, 0.4)}
xgb_model = xgb.XGBRegressor(objective="reg:linear",random_state=None)
time_split = TimeSeriesSplit(n_splits = 2)
#time_split = KFold(n_splits = 3)
#xgb_search = RandomizedSearchCV(xgb_model,param_distributions=params, random_state=2, n_iter=4, cv=time_split, verbose=1, n_jobs=1, return_train_score=True)
xgb_search = RandomizedSearchCV(xgb_model,param_distributions=params, random_state=None, n_iter=4, cv=time_split, verbose=1, n_jobs=1, return_train_score=True)
xgb_search.fit(X_train,y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[09:32:54] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[09:33:11] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[09:34:00] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[09:34:09] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[09:34:26] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[09:34:40] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[09:35:21] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[09:35:48] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squ

RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_w...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6f0e14ef90>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6f0e17d1d0>,
                     

              Year  Country Name   Series Code
1077728 1977-01-01  2.900363e+08  9.886577e+01
1886092 1981-01-01  7.186466e+09  9.320895e+10
1383323 1979-01-01  4.000409e+11  2.864232e+07
1075330 1977-01-01  6.956189e+08 NaN          
3073395 1987-01-01  9.584080e+09 -2.553987e+08
...            ...           ...           ...
4304572 1994-01-01  1.429246e+10  1.185109e+13
1692743 1980-01-01  1.517425e+10  1.409230e+09
6550634 2005-01-01  1.480606e+09  6.671851e+01
6423388 2004-01-01  2.989822e+08  4.697937e+01
6413414 2004-01-01  5.001289e+10  6.004062e+01

[2051721 rows x 3 columns]


In [329]:
y_pred = xgb_search.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
print ('RMSE:', rms)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').